# IAB딥러닝 9월 13일 
> Overview - 추천시스템, 텍스트분석

- toc:true
- branch: master
- badges: true
- comments: true
- author: 김준형

## Import

In [1]:
from fastai.collab import *  # 추천시스템
from fastai.text.all import *  # 텍스트분석

In [2]:
import pandas as pd

### fastai를 이용한 분석 단계

| |이미지분석(CNN) | 추천시스템 | 텍스트분석 | GAN | 
|:-:|:-:|:-:|:-:|:-:|
|1단계|ImageDataLoaders|CollabDataLoaders| TextDataLoaders | DataBlock -> dls |  
|2단계|cnn_learner()|collab_learner()| language_model_learner() | GANLearner.wgan() | 
|3단계|lrnr.fine_tune(1)|lrnr.fit()| lrnr.fit() | lrnr.fit() |
|4단계|lrnr.predict(), lrnr.model(X)|lrnr.model(X)| lrnr.predict() | | 

## 추천시스템

### 1단계

In [3]:
df_view = pd.read_csv('https://raw.githubusercontent.com/guebin/DL2022/master/_notebooks/2022-09-08-rcmd_view.csv')
df_view

,커피1,커피2,커피3,커피4,커피5,커피6,커피7,커피8,커피9,커피10,홍차1,홍차2,홍차3,홍차4,홍차5,홍차6,홍차7,홍차8,홍차9,홍차10
0,4.149209,NaN,NaN,4.078139,4.033415,4.071871,NaN,NaN,NaN,NaN,1.142659,1.109452,NaN,0.603118,1.084308,NaN,0.906524,NaN,NaN,0.903826
1,4.031811,NaN,NaN,3.822704,NaN,NaN,NaN,4.071410,3.996206,NaN,NaN,0.839565,1.011315,NaN,1.120552,0.911340,NaN,0.860954,0.871482,NaN
2,4.082178,4.196436,NaN,3.956876,NaN,NaN,NaN,4.450931,3.972090,NaN,NaN,NaN,NaN,0.983838,NaN,0.918576,1.206796,0.913116,NaN,0.956194
3,NaN,4.000621,3.895570,NaN,3.838781,3.967183,NaN,NaN,NaN,4.105741,1.147554,NaN,1.346860,NaN,0.614099,1.297301,NaN,NaN,NaN,1.147545
4,NaN,NaN,NaN,NaN,3.888208,NaN,3.970330,3.979490,NaN,4.010982,NaN,0.920995,1.081111,0.999345,NaN,1.195183,NaN,0.818332,1.236331,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.511905,1.066144,NaN,1.315430,NaN,1.285778,NaN,0.678400,1.023020,0.886803,NaN,4.055996,NaN,NaN,4.156489,4.127622,NaN,NaN,NaN,NaN
96,NaN,1.035022,NaN,1.085834,NaN,0.812558,NaN,1.074543,NaN,0.852806,3.894772,NaN,4.071385,3.935935,NaN,NaN,3.989815,NaN,NaN,4.267142
97,NaN,1.115511,NaN,1.101395,0.878614,NaN,NaN,NaN,1.329319,NaN,4.125190,NaN,4.354638,3.811209,4.144648,NaN,NaN,4.116915,3.887823,NaN
98,NaN,0.850794,NaN,NaN,0.927884,0.669895,NaN,NaN,0.665429,1.387329,NaN,NaN,4.329404,4.111706,3.960197,NaN,NaN,NaN,3.725288,4.122072


In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/DL2022/master/_notebooks/2022-09-08-rcmd_anal.csv')
df

,user,item,rating,item_name
0,1,15,1.084308,홍차5
1,1,1,4.149209,커피1
2,1,11,1.142659,홍차1
3,1,5,4.033415,커피5
4,1,4,4.078139,커피4
...,...,...,...,...
995,100,18,4.104276,홍차8
996,100,17,4.164773,홍차7
997,100,14,4.026915,홍차4
998,100,4,0.838720,커피4


In [6]:
df.item.unique(), df.user.unique()  # unique()는 데이터에 고유값들이 어떠한 종류들이 있는지 알고 싶을 때 사용하는 함수

(array([15,  1, 11,  5,  4, 14,  6, 20, 12, 17,  8,  9, 13, 19, 18, 16,  2,
         3, 10,  7], dtype=int64),
 array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100], dtype=int64))

In [9]:
dls = CollabDataLoaders.from_df(df)  #협업 필터링에 적합한 항목 생성

In [10]:
dls.show_batch()

,user,item,rating
0,78,13,4.020114
1,14,1,4.329083
2,52,16,4.008471
3,79,15,4.105639
4,16,14,0.946549
5,98,11,4.125190
6,12,9,4.502565
7,80,13,3.725410
8,5,10,4.010983
9,76,2,0.725603


In [11]:
X, y = dls.one_batch()

In [12]:
X[0], y[0]  # 60번 유저가 3번 아이템을 먹었을 때, 평점 1.1309

(tensor([60,  3]), tensor([1.1309]))

### 2단계

In [13]:
lrnr = collab_learner(dls, y_range = (0, 5))

In [15]:
lrnr.fit(30)

epoch,train_loss,valid_loss,time
0,0.982962,0.960218,00:00
1,0.854534,0.739310,00:00
2,0.728671,0.549827,00:00
3,0.612306,0.398833,00:00
4,0.508047,0.283823,00:00
5,0.416964,0.201994,00:00
6,0.340235,0.146098,00:00
7,0.277236,0.109585,00:00
8,0.226001,0.086219,00:00
9,0.185047,0.071640,00:00


### 4단계

In [16]:
yhat = lrnr.model(X.to("cuda:0"))
yhat

tensor([1.0497, 1.0491, 3.8868, 1.0842, 1.0308, 3.9974, 3.9883, 3.9073, 3.8864,
        3.9928, 1.0348, 4.0798, 1.0885, 0.9187, 4.1383, 4.0982, 4.1652, 3.8058,
        0.9393, 4.0877, 4.0433, 1.0097, 0.8921, 0.8991, 4.1124, 0.9828, 1.0059,
        1.0155, 0.9488, 0.9874, 3.9683, 1.0021, 1.0736, 0.9726, 0.9243, 0.9903,
        4.0987, 3.9215, 4.0557, 0.8602, 3.8443, 4.0904, 4.0177, 0.9665, 1.0474,
        4.0748, 1.1496, 0.9478, 4.0791, 1.1091, 4.0488, 1.0230, 4.0360, 4.0741,
        0.7673, 3.9696, 1.0462, 3.9625, 4.1699, 1.0259, 4.0511, 3.9820, 0.9829,
        3.9767], device='cuda:0', grad_fn=<AddBackward0>)

In [17]:
X.shape

torch.Size([64, 2])

In [21]:
X[0:1]

tensor([[60,  3]])

In [19]:
lrnr.model(X[0:1].to("cuda:0"))

tensor([1.0497], device='cuda:0', grad_fn=<AddBackward0>)

`-` 새로운 데이터 생성 후 예측

In [22]:
Xnew = torch.tensor([[1, 2]])

In [23]:
lrnr.model(Xnew.to("cuda:0"))

tensor([3.9995], device='cuda:0', grad_fn=<AddBackward0>)

## 텍스트분석

### 1단계

In [25]:
df = pd.DataFrame({'text' : ['h e l l o . h e l l o ! h e l l o ? h e l l o !! h e l l o ??'] * 20000})
df

,text
0,h e l l o . h e l l o ! h e l l o ? h e l l o !! h e l l o ??
1,h e l l o . h e l l o ! h e l l o ? h e l l o !! h e l l o ??
2,h e l l o . h e l l o ! h e l l o ? h e l l o !! h e l l o ??
3,h e l l o . h e l l o ! h e l l o ? h e l l o !! h e l l o ??
4,h e l l o . h e l l o ! h e l l o ? h e l l o !! h e l l o ??
...,...
19995,h e l l o . h e l l o ! h e l l o ? h e l l o !! h e l l o ??
19996,h e l l o . h e l l o ! h e l l o ? h e l l o !! h e l l o ??
19997,h e l l o . h e l l o ! h e l l o ? h e l l o !! h e l l o ??
19998,h e l l o . h e l l o ! h e l l o ? h e l l o !! h e l l o ??


In [26]:
dls = TextDataLoaders.from_df(df, text_col = 'text', is_lm = True)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck


In [27]:
dls.show_batch()

,text,text_
0,xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o,h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o .
1,? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l,xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o
2,? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l,? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l
3,o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e,? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l
4,l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h,o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e
5,l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos,l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h
6,e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ?,l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos
7,h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ?,e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ?
8,! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o,h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ? ? xxbos h e l l o . h e l l o ! h e l l o ? h e l l o ! ! h e l l o ?


### 2단계

In [28]:
lrnr = language_model_learner(dls, AWD_LSTM)

In [31]:
lrnr.fit(20)

epoch,train_loss,valid_loss,time
0,0.953486,0.372985,00:13
1,0.617590,0.249084,00:13
2,0.475271,0.214262,00:13
3,0.382338,0.186492,00:13
4,0.365782,0.170414,00:13
5,0.324649,0.150192,00:13
6,0.322575,0.135068,00:13
7,0.297149,0.119661,00:13
8,0.268460,0.103948,00:13
9,0.236526,0.085444,00:13


In [33]:
lrnr.predict('h ', n_words = 30)

'h e l l o . h e l l o ! h e l l o ? ? h e l l o . h e l l o'

`-` IAB 딥러닝 수업 자료를 기반으로 공부한 내용입니다.